## Creating County Files from the Microsoft US Building Footprints Dataset


Microsoft recently made freely available a dataset of [building footprints](https://github.com/Microsoft/USBuildingFootprints) for the entire USA. Thank you Microsoft! These footprints were generated using machine learning on satellite imagery. Although not perfect - buildings may be missing or imperfectly represented - this is a unique and valuable dataset that attempts to map every building in the US. It’s also really cool to visualize and explore. You can read more about the data on the [Bing Blog](https://blogs.bing.com/maps/2018-06/microsoft-releases-125-million-building-footprints-in-the-us-as-open-data) and in this [New York Times article](https://www.nytimes.com/interactive/2018/10/12/us/map-of-every-building-in-the-united-states.html), the latter of which includes some great maps of the data. 

On the [Github site](https://github.com/Microsoft/USBuildingFootprints) the data are made available as [GeoJSON](https://geojson.org) files, one per state. For some states like CA, FL, IL, MI, NY, PA, TX, OH, and a few others, these files are huge - 1 GB or larger.  These large files are extremely difficult to work with on your average personal computer, whether with desktop software like ArcGIS or QGIS or programmatically in R or Python. Moreover, many folks who work with geospatial data have more experience working with ESRI Shapefiles rather than GeoJSON data.

Consequently, there are a number of web posts discussing the desire for and ways to wrangle these data into smaller county based files. This is also a task that I was asked to help out with to make these data more widely available to the UC Berkeley campus community. 

Unfortnately, none of the suggested approaches for splitting the data into county files worked for me. Consequently, I took advantage of some time off due to a broken ankle and came up with an approach that leverages [PostGIS](https://postgis.net/), the geospatial extension to the fantastic free and open source database software [PostgreSQL](https://www.postgresql.org/). 

What I liked about this approach is that it gave me an opportunity to get reacquainted with PostGIS, a tool that I used all the time in my previous position but haven't used for a while. It's a great tool for scaling up geospatial data management and analysis. As a new twist, I brought Python to the party and interacted with PostGIS using the [psycopg2](http://initd.org/psycopg/docs/) database adapter.

My steps are detailed in this notebook. It ain't pretty and nor is it perfect but it works! That said, this is a draft and improvements to the process are sure to be needed. You can check out the processing results for California counties in this [Google drive folder](https://drive.google.com/drive/folders/1-XGvS25tQKKQ3HTqWjAfLJ4PaeXJ9yyY?usp=sharing).  

Given that this is the holiday season, I call it the **ugly sweater** of workflows. Feel free to send me an email if you have suggested improvements for any of the steps!
- Patty Frontiera, [D-Lab](https://dlab.berkeley.edu), UC Berkeley (pattyf@berkeley.edu)

<img width="600px" src="./ugly_sweater.png"></img>


In [ ]:
# Import the libraries we will use
import psycopg2
import numpy as np
import pandas as pd
import geopandas as gpd

import os.path


## Step 1. Census TIGER/Line data for US Counties

Download the TIGER/Line boundary file for all US Counties from the Census website. 

Do not use the Census Cartographic Boundary files as you may miss some footprints at boundaries of states / census tracts.

### Loading County TIGER/Line Shapefile into Postgis
This step was done manually. Could automate in this notebook. The steps were:

Things to do to make `counties_sub` table:
- Download the US County TIGER/Line shapefile (tl_2018_us_county.shp) from the [Census Website](https://www.census.gov/geo/maps-data/data/tiger.html
- Open the shapefile in QGIS or ArcGIS to check it and then export to a Shapefile or GeoJSON file, setting the output CRS to WGS84 (by default census data are NAD83).
- Use pgsql2shp or ogr2ogr to load the county shapefile/GeoJSON file into postgis.
- Explicitly set the CRS of the counties table in PostGIS to 4326
- Create a GIST (geo index) on the counties file
- Use St_subdivide to make a new table `counties_sub` that will be used for more efficient spatial intersection queries.

## Step 2. Read in the County Boundary Data as a Geopandas dataframe.

In [ ]:
uscounties = gpd.read_file("data/tl_2018_us_county/tl_2018_us_county.shp")
uscounties = uscounties[["STATEFP","COUNTYFP","GEOID","NAME","geometry"]]

uscounties.head()

In [ ]:
# Summary Stats on the counties
print("The number of counties in the US Counties file is: ", len(uscounties.index))
print("The number of distinct states in the US Counties File is: ", len(set(uscounties.STATEFP)))

### Note:
If the number of states (plus DC) in the US Counties file is greater than **51** then the file includes the counties or county equivalents in US Territories. IF the Microsoft Buildgings Footprints does not contain data for these places then we should remove those rows from the uscounties geopandas dataframe.

<blockquote>
The United States of America is a federal republic consisting of 50 states, a federal district (Washington, D.C., the capital city of the United States), five major territories, and various minor islands. - Wikipedia
</blockquote>

## Step 4. Read in a table of all the State Building Footprints you want to process.

This table was created from the table on the [Microsoft Building footprints Github site](https://github.com/Microsoft/USBuildingFootprints) and imported into [this Google Sheet](https://docs.google.com/spreadsheets/d/1wEk1DC-B4AjKdAUJ-9QtcTv7IIPlTvEhuX2HV2o01SI/edit?usp=sharing). You can download it as a CSV file to use with this script.

- If you don't want to process the entire USA, delete any states you don't want to process from the CSV file or delete the rows from the dataframe after you import.


In [ ]:
sfips_lookup = pd.read_csv("data/USA_statefips_lookup.csv", dtype={"STATE": str, "FIPS": str, "ABBREV": str, 'BLDG_GEOJSON': str})
sfips_lookup.head(2)

In [ ]:
# Chek the number of states/geojson files to be processed.
print("The Microsoft BuildingFootprints dataset includes footprints for ", len(sfips_lookup.index), " states plus DC.")
print("The total number of footprints in the dataset is: ", sum(sfips_lookup['BLDG_COUNT']))

In [ ]:
#sfips_lookup

## Step 5. Download & unzip all state bldg footprint files from

- Go to https://github.com/Microsoft/USBuildingFootprints, find the links to the state GeoJSON footprint files and download the ones you want to process - manually or via a script, curl, wget etc.  

- Once they are downloaded you can execute a simple command in the folder in which the downloaded files reside to unzip them all:

<pre>
for file in *.zip; do unzip $file; done
</pre>

**We could do this in the notebook.**

## Step 6. Load the GeoJSON files into PostGIS

- Use the code below to create a **shell script** (`load_geojson_ogr.sh`) to load all unzipped geojson files into PostGIS database tables.

- If you are not processing all states, subset the `sfips_lookup` dataframe.
    
- Change the permissions on the shell script file (`chmod 755 load_geojson_ogr.sh`) to make it executable.
- Run the script (./load_geojson_ogr.sh) from the directory in which the geojson files reside

**Suggested update not used by this process - read in only polygon geometry by adding:** -where "OGR_GEOMETRY='POLYGON or MULTI'"

In [ ]:
f = open('load_geojson_ogr.sh', 'w')

for index, row in sfips_lookup.iterrows():
    jfile =row["BLDG_GEOJSON"]
    scode = row['ABBREV']
    tname = scode + "_footprints"
    #print(scode, ":", jfile , ": ", tname)
    f.write('ogr2ogr -f "PostgreSQL" PG:"dbname=pattyf user=pattyf" ' + jfile + ' -nln ' + tname +  ' -a_srs "EPSG:4326"\n')

f.close()

### Did you run the script `./load_geojson_ogr.sh` ?

This is likely the longest part of the process, takes ~ 5-10 hours I estimate.

- I started it around 9pm and at 12:30am still 24 states to go (Nevada - Wyoming).

- You may want to split this into 2-4 scripts - my computer is starting to overheat...


## Step 7. Create a function to Subdivide each State Footprints table.

Create a function that will use the PostGIS function `ST_Subdivide` to subdivide each of the state footprints tables to improve the performance of spatial intersection queries. 

This step was informed by this [Paul Ramsey blog post](http://blog.cleverelephant.ca/2017/12/postgis-scaling.html).

Here are the steps that will be needed, with an example using Rhode Island (RI) data:

<pre>
-- Create a new subdivided table
CREATE TABLE ri_footprints_sub AS
SELECT ogc_fid, ST_Subdivide(wkb_geometry) AS geom
FROM ri_footprints;

-- Update the SRID on each new sub table
select UpdateGeometrySRID('ri_footprints_sub','geom',4326);

--Create the GIST on each new sub table
CREATE INDEX ri_footprints_sub_geom_idx ON ri_footprints_sub USING GIST ( geom );
</pre>


The function to automate the first step is shown below.

In [ ]:
# Create the function within postgresql to create a sub table
CREATE OR REPLACE FUNCTION create_state_sub_table(tsub_name varchar(30), t_name varchar(30))
  RETURNS VOID AS
$func$
BEGIN

EXECUTE format('
        CREATE TABLE IF NOT EXISTS %I AS
            SELECT ogc_fid, ST_Subdivide(wkb_geometry) AS geom
            FROM %I;', tsub_name, t_name);

END
$func$ LANGUAGE plpgsql;

## Step 8. Implement the function over all states.

For example:
<pre>
create_state_sub_table('al_footprints_sub', 'al_footprints')
</pre>


In [ ]:
# Implement function create_state_sub_table over all state footprints tables.
try:
    conn = psycopg2.connect("dbname='pattyf' user='pattyf' host='localhost' password='XXXXXXX'")
    print("connected")

    for index, row in sfips_lookup.iterrows():
        scode = row['ABBREV'].lower()
        sub_table_name = scode + "_footprints_sub"   
        table_name = scode + "_footprints"
 
        #create_state_sub_table(sub_table_name, table_name)
        #create_state_sub_table('al_footprints_sub', 'al_footprints')
        q1 = """SELECT create_state_sub_table('{sname}', '{tname}');""".format(sname=sub_table_name, tname=table_name)

        print(q1)

        try:
            cur = conn.cursor()
            print('about to execute query')
            cur.execute(q1)
            print('about to commit query results')
            conn.commit()
            # close the database communication
            cur.close()

        except psycopg2.DatabaseError as error:
            print(error)

except:
    print("I am unable to connect to the database")

finally:
    if conn is not None:
        conn.close()

print("Back from Postgis")


## Step 9. Set the SRID on all the sub tables

For example:

<pre>
-- Update the SRID
select UpdateGeometrySRID('ri_footprints_sub','geom',4326);
</pre>

*Could add this to the `create_state_sub_table` function.*

In [ ]:
# Upate SRID on all sub tables
try:
    conn = psycopg2.connect("dbname='pattyf' user='pattyf' host='localhost' password='XXXXXX'")
    print("connected")

    for index, row in sfips_lookup.iterrows():
        scode = row['ABBREV'].lower()
        sub_table_name = scode + "_footprints_sub"# Update the SRID on the sub table    
        
        # select UpdateGeometrySRID('ri_footprints_sub','geom',4326);
        q1 = """select UpdateGeometrySRID('{name}','geom',4326);""".format(name=sub_table_name)

        print(q1)

        try:
            cur = conn.cursor()
            print('about to execute query')
            cur.execute(q1)
            print('about to commit query results')
            conn.commit()
            # close the database communication
            cur.close()

        except psycopg2.DatabaseError as error:
            print(error)

except:
    print("I am unable to connect to the database")

finally:
    if conn is not None:
        conn.close()

print("Back from Postgis")


## Step 10. Create the GIST on each of the sub table

For example:
<pre>
SELECT CREATE INDEX ri_footprints_sub_geom_idx ON ri_footprints_sub USING GIST ( geom );

</pre>


This step takes about 1 - 4 minutes per state, depending on table size.

*Could add this to the `create_state_sub_table` function.*

In [ ]:
# Create GIST on all sub tables
try:
    conn = psycopg2.connect("dbname='pattyf' user='pattyf' host='localhost' password='XXXXXXX'")
    print("connected")

    for index, row in sfips_lookup.iterrows():
        scode = row['ABBREV'].lower()
        subtable_name = scode + "_footprints_sub"# Update the SRID on the sub table 
        subtable_index = subtable_name +  '_geom_idx'
        
        # select UpdateGeometrySRID('ri_footprints_sub','geom',4326);
        q1 = """CREATE INDEX {stable_index} on {stable} USING GIST (geom);""".format(stable_index = subtable_index, stable=subtable_name)

        print(q1)

        try:
            cur = conn.cursor()
            print('about to execute query')
            cur.execute(q1)
            print('about to commit query results')
            conn.commit()
            # close the database communication
            cur.close()

        except psycopg2.DatabaseError as error:
            print(error)

except:
    print("I am unable to connect to the database")

finally:
    if conn is not None:
        conn.close()

print("Back from Postgis")


## Step 11. - OPTIONAL. Make sure all our tables have valid geometry

The command to do this looks like this:

<pre>
UPDATE counties_sub SET geom = ST_MakeValid(geom) WHERE NOT ST_IsValid(geom);
</pre>

*This step can take a several hours ~ 5!*

**NOTE** Try without this step now that we are doing point in polygon intersection rather than polygon-polygon (which was throughing errors that this didn't resolve anyway).

In [ ]:
# Make sure all sub tables have valid geometry
try:
    conn = psycopg2.connect("dbname='pattyf' user='pattyf' host='localhost' password='XXXXXXX'")
    print("connected")

    for index, row in sfips_lookup.iterrows():
        scode = row['ABBREV'].lower()
        subtable_name = scode + "_footprints_sub"# Update the SRID on the sub table 
        
        # select UpdateGeometrySRID('ri_footprints_sub','geom',4326);
        q1 = """UPDATE {stable} SET geom = ST_MakeValid(geom) WHERE NOT ST_IsValid(geom);""".format(stable=subtable_name)

        print(q1)

        try:
            cur = conn.cursor()
            print('about to execute query')
            cur.execute(q1)
            print('about to commit query results')
            conn.commit()
            # close the database communication
            cur.close()

        except psycopg2.DatabaseError as error:
            print(error)

except:
    print("I am unable to connect to the database")

finally:
    if conn is not None:
        conn.close()

print("Back from Postgis")


## Step 12. Create function to add centroid geometry to all county tables.

Create function to add centroid geometry to each state footprints subdivided table. This is an indexing operation that will speed up spatial intersections.

What this step means is as follows: a county footprints table will contain the footprints for all state footprint centroids that intersect the county boundary polygon.

#### Cautions:

- Footprints in a county file whose centroid is not in the county could be dropped by this process if the county borders another state (because we are only querying against the state footprints). If it borders a county in the same state the footprint would be reallocated to the other county (which is not necessarily a problem).




In [ ]:
# Create the function within postgresql to create a sub table
CREATE OR REPLACE FUNCTION create_state_centroids(tname varchar(30), gindex_name varchar(64))
  RETURNS VOID AS
$func$
BEGIN

-- Create a new geometry column to hold the centroid_geom
EXECUTE format('
        SELECT AddGeometryColumn (''public'', ''%I'', ''centroid_geom'', 4326, ''POINT'', 2);', tname);

-- Populate the centroid_geom column
EXECUTE format('
        UPDATE %I SET centroid_geom = ST_Centroid(geom);', tname);
               
--Create the index on the centroid_geom
EXECUTE format('
        CREATE INDEX %I ON %I USING GIST (centroid_geom);', gindex_name, tname );

END
$func$ LANGUAGE plpgsql;

## Step 13.  Apply the create_state_centroids function

WARNING: This took about 5 hours for all 50 states.

In [ ]:
# Populate centroid_geom
try:
    conn = psycopg2.connect("dbname='pattyf' user='pattyf' host='localhost' password='XXXXXXX'")
    print("connected")

    for index, row in sfips_lookup.iterrows():
        scode = row['ABBREV'].lower()
        subtable_name = scode + "_footprints_sub"# Update the SRID on the sub table 
        subtable_index = scode + "_footprints_sub_centroid_geom_idx"# Update the SRID on the sub table
        
        # The Query to update the tables with centroid geom
        q1 = """SELECT create_state_centroids('{sname}', '{gindex}');""".format(sname=subtable_name, gindex=subtable_index)

        print(q1)

        try:
            cur = conn.cursor()
            print('about to execute query')
            cur.execute(q1)
            print('about to commit query results')
            conn.commit()
            # close the database communication
            cur.close()

        except psycopg2.DatabaseError as error:
            print(error)

except:
    print("I am unable to connect to the database")

finally:
    if conn is not None:
        conn.close()

print("Back from Postgis")


## Step 14. Create the County Footprint Tables

Spatial intersection queries with the sub tables.

**This took about 40 minutes for all counties in all states.** Fast because we first subdivided the data.

The query returns building footprint polygons whose centroid intersects the boundary or interior of a county polygon.

The output will be one table for each county in each state for entire USA!

---

### The query looks like this:
<pre>
CREATE TABLE IF NOT EXISTS {county_tname} as
SELECT
  c.name,
  f.geom AS the_geom
FROM
    counties_sub c,
    {state_footprints_sub} f
WHERE
    c.geoid = '{geoid}' AND
    ST_Intersects(c.geom, f.centroid_geom);""".format(county_tname=ctable_name, state_footprints_sub=subtable_name, geoid=the_geoid)

</pre>



In [ ]:
# Create County footprints tables
# This took 2 hours?
# Data Frame to keep track of processing notes
proc_errors_df = pd.DataFrame(columns = ['state', 'county', 'geoid','error'])

try:
    conn = psycopg2.connect("dbname='pattyf' user='pattyf' host='localhost' password='XXXXXXX'")
    print("connected")

    for index, row in sfips_lookup.iterrows():
        scode = row['ABBREV'].lower()
        sfips = row['FIPS']
        subtable_name = scode + "_footprints_sub" 
        
        print("Processing : ", scode, " With FIPS: ", sfips)
        state_county_df = uscounties[uscounties['STATEFP']==sfips]
        
        for item, row in state_county_df.iterrows():
            the_geoid = row['GEOID']
            the_county = row['NAME']
            ctable_name = scode + "_" + the_geoid + "_footprints"
            print("About to create table: ", ctable_name)
            
            q1 = """
CREATE TABLE IF NOT EXISTS {county_tname} as
SELECT
  c.name,
  f.geom AS the_geom
FROM
    counties_sub c,
    {state_footprints_sub} f
WHERE
    c.geoid = '{geoid}' AND
    ST_Intersects(c.geom, f.centroid_geom);""".format(county_tname=ctable_name, state_footprints_sub=subtable_name, geoid=the_geoid)
        
            #print(q1)

            try:
                cur = conn.cursor()
                print('about to execute query')
                cur.execute(q1)
                print('about to commit query results')
                conn.commit()
                # close the database communication
                cur.close()

            except psycopg2.DatabaseError as error:
                print("PROBLEM executing the query....")
                print(error)
                proc_errors_df.loc[len(proc_errors_df)]=[scode, the_county, the_geoid, error]
             
                print("Closing the CURSOR")
                cur.close()
                print("Closing the CONN")
                conn.close()
                
                try:
                    print("GONNA RESTART THE CON....")
                    conn = psycopg2.connect("dbname='pattyf' user='pattyf' host='localhost' password='XXXXXXX'")
                    print("connected")
                except psycopg2.DatabaseError as error:
                    print("CANT RESTART CONN")
                    print(error)
                

except:
    print("I am unable to connect to the database")

finally:
    if conn is not None:
        print("Closing DB Connection.")
        conn.close()

print("Back from Postgis")
#Save processing errors
proc_errors_df.to_csv("bldg_footprints_proc_errors3.csv", index=False)

**Note**: no processing errors were recorded for the above operation.

## Step 15. Output the county tables to shapefiles

Create a shell script to dump the PostGIS county tables to CSV files containing the footprint geometry as WKT (well-known text). We do this becuse of shapefile limitations & because the output will be smaller & compress well compared to geojson files.

For example:
<pre>
ogr2ogr -f "CSV" -lco GEOMETRY=AS_WKT outfile.csv PG:"host=localhost dbname=pattyf user=pattyf port=5432" county_footprints_table
</pre>

In [ ]:
# Create a scriptfile to ouput all county footprint tables to CSV
# CSV smaller than GeoJSON 
# Shapefiles have size limits & will truncate the data
f = open('export_county_footprint_to_csv.sh', 'w')

for index, row in sfips_lookup.iterrows():
    scode = row['ABBREV'].lower()
    sfips = row['FIPS']
    subtable_name = scode + "_footprints_sub" 

    print("Processing : ", scode, " With FIPS: ", sfips)
    state_county_df = uscounties[uscounties['STATEFP']==sfips]

    for item, row in state_county_df.iterrows():
        the_geoid = row['GEOID']
        the_county = row['NAME']
        ctable_name = scode + "_" + the_geoid + "_footprints"
        print("About to write cmd for table: ", ctable_name)
        out_csvfile = ctable_name + ".csv"
        f.write('ogr2ogr -f "CSV" -lco GEOMETRY=AS_WKT ' + out_csvfile + ' PG:"host=localhost dbname=pattyf user=pattyf port=5432" ' + ctable_name + '\n')
f.close()

## Step 16. Run the script to create county shapefiles

Move this script into an empty directory first!

Make the script executeable (chmod 755 export_county_footprint_to_csv.sh).

Run the script (./export_county_footprint_to_csv.sh) 

Wait for it to finish - takes about an hour for all counties for all states.

Check the results...

- Is there a shp file for each county in the 50 states plus DC?

- Do the number of features in the county tables add up to the total in the states tables and in the table on the MS github website (125,192,184)?



## Step 17. Check the Results

Do we have one footprint file/PostGIS table per county? 

- If not, why not? Only reason should be because no features in that county (unlikely)?

Do the number of buildings listed on the Microsoft USBuildingFootprints github page match the number of building footprints in the output csv files (and PostGIS tables)? If yes done!


### How many building footprints in total are in the MS BuildingFootprints files?

In [ ]:
#ms_website_total = 125192184
msbld_count= sum(sfips_lookup['BLDG_COUNT'])
msbld_count


### How many footprints are in the output county files?

We can use **wc -l *.csv > county_feature_counts.csv** to create a file of counts and counties that we can read into a pandas df. Note we did a little post-processing of line_count.csv to remove leading zeros, not county csv files, and add headers.


In [ ]:
# Read in county feature counts
fcounts_df =  pd.read_csv("county_feature_counts.csv", sep="|")

In [ ]:
fcounts_df.head()

In [ ]:
f_count=sum(fcounts_df['count'])  # should be 125192184
print("Number of footprints in the county ouput files: ", f_count)

### What is the difference in the total MS Building count & the number of footprints in the output county files?

- What amount of difference is acceptable?

In [ ]:
the_diff = msbld_count - f_count
if the_diff > 0:
    print("There are ", the_diff, "FEWER footprints in the county ouput files.")
elif the_diff < 0:
     print("There are ", the_diff, "MORE footprints in the county ouput files.")
else:
    print("There are the same number of footprints in the county output files.")

In [ ]:
#How Many counties were processed?
processed_counties = len(fcounts_df.index)
print("The number of US Counties processed: ", processed_counties )

# What is the number of counties in the US States plus DC?
# American Samoa (60), Guam (66), Northern Mariana Islands(69),
# Puerto Rico (72), Virgin Islands of the U.S. (78)
territories = ['60', '66','69','72','78']
uscounties_51 = uscounties[~uscounties['STATEFP'].isin(territories)]
total_counties51 = len(uscounties_51.index)
print("Total number of counties in the 50 US states plus DC: ", total_counties51)

if total_counties51 == processed_counties:
    print("The number of counties processed and output to file equals the total number of US State counties plus DC.")

### Create a table summarizing the number of footprints output per state.

This will summarize the discrepancies between the input state footprints files and the output county files. It's up to the individual user to assess whether any differences are acceptable for the application at hand.

In [ ]:
#Add GEOID to fcounts_df
#add state fips to fcounts_df
#add fcounts to sfips_lookup.df
# see what states have big mismatches to identify scope of probelm
# also figure out why some counties have zero features

In [ ]:
fcounts_df['state'] = fcounts_df['countyfile'].str.split('_').str[0]
#fcounts_df.head()

In [ ]:
fcounts2 =fcounts_df.groupby(['state']).sum()
fcounts2.reset_index(level=0, inplace=True)
fcounts2['state'] = fcounts2['state'].str.upper()
fcounts2.head()

In [ ]:
len(fcounts2.index) == len(sfips_lookup.index)

In [ ]:
# Merge the files
sfips2 =sfips_lookup.merge(fcounts2,left_on='ABBREV', right_on='state', how='outer')
#sfips2  # This is our summary stats for ouput

In [ ]:
# Check the table totals
print( "Total foootprint count: ", sum(sfips2['BLDG_COUNT']))
print(" County footprint count: ", sum(sfips2['count']))
print("Count differences: ", sum(sfips2['BLDG_COUNT']) - sum(sfips2['count']))

In [ ]:
# Add the differences to the table
sfips2['count_diff'] = sfips2['BLDG_COUNT'] - sfips2['count']
sfips2.sort_values(by=['count_diff'], inplace=True,  ascending=False)
sfips2.head()

In [ ]:
# Write processing metadata to file
sfips2.to_csv("Output_counts_by_county_and_state.csv", index=False)

## Now zip all the csv files!

<pre>
for file in *.csv; do zip ${file%.*}.zip $file; done
</pre>

Hey did you know that these zipped files can be read directly into QGIS?

### CRS for all files

The CRS (coordinate reference system) or projection or SRS for all output files is EPSG: 4326 or WGS84.


## Step 17. Cleanup

Delete all files you don't need!

Share output so folks don't need to repeat this wheel!

# Done

You can access the output files from this processing for the state of California [here](https://drive.google.com/drive/folders/1-XGvS25tQKKQ3HTqWjAfLJ4PaeXJ9yyY?usp=sharing).

---
Last updated December 20, 2018